In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
file_path = '.out/master_data.json'

In [3]:
file_path = '.out/master_data.json'
try:
    with open(file_path, 'r') as file:
        existing_data = json.load(file)
except FileNotFoundError:
    existing_data = []

In [4]:
try:
    with open(file_path, 'r') as file:
        existing_data = json.load(file)
except FileNotFoundError:
    existing_data = []
    
def count_diplotype(masterdata) -> list:
        data = masterdata
        result_list = []
        
        # Loop through each sample's gene details
        for sample in data:
            gender = sample['gender']
            for gene_entry in sample['gene_detail']:
                gene = gene_entry['gene']
                diplotype_list = gene_entry['diplotype']
                
                number_diplotype = len(diplotype_list)
                
                count_dict = {}
                
                # Count occurrences of each diplotype
                for diplotype in diplotype_list:
                    if number_diplotype <=1:
                        if diplotype in count_dict:
                            count_dict[diplotype] += (1 * (1 / number_diplotype))
                        else:
                            count_dict[diplotype] = (1 * (1 / number_diplotype))
                    else:
                        count_dict[diplotype] = np.nan
        
                # Convert the count_dict into a list of dictionaries
                for diplotype, count in count_dict.items():
                    result_list.append({'gene': gene, 'gender': gender, 'diplotype': diplotype, 'diplotype_count': count})
        
        # Create a DataFrame from the result_list
        result_df = pd.DataFrame(result_list)
                    
        # Group and aggregate the results using Pandas
        result_all = (
            pd.DataFrame(result_list)
            .dropna()
            .groupby(['gene', 'gender', 'diplotype'])
            .sum()
            .reset_index()
            .sort_values(by="gene")
            .to_dict(orient='records')
        )            
        
        result_male = result_df[result_df['gender'] == 'male']
        result_female = result_df[result_df['gender'] == 'female']
        
        result_by_gender = {
            'all': result_all,
            'male': result_male.to_dict(orient='records'),
            'female': result_female.to_dict(orient='records')
        }
        
        return result_by_gender
    
data = count_diplotype(existing_data)



In [5]:
def count_diplotype(masterdata) -> list:
        data = masterdata
        result_list = []
        
        # Loop through each sample's gene details
        for sample in data:
            gender = sample['gender']
            for gene_entry in sample['gene_detail']:
                gene = gene_entry['gene']
                diplotype_list = gene_entry['diplotype']
                
                number_diplotype = len(diplotype_list)
                
                count_dict = {}
                
                # Count occurrences of each diplotype
                for diplotype in diplotype_list:
                    if number_diplotype <=1:
                        if diplotype in count_dict:
                            count_dict[diplotype] += (1 * (1 / number_diplotype))
                        else:
                            count_dict[diplotype] = (1 * (1 / number_diplotype))
                    else:
                        count_dict[diplotype] = np.nan
        
                # Convert the count_dict into a list of dictionaries
                for diplotype, count in count_dict.items():
                    result_list.append({'gene': gene, 'gender': gender, 'diplotype': diplotype, 'diplotype_count': count})
        
        # Create a DataFrame from the result_list
        result_df = pd.DataFrame(result_list)
                    
        # Group and aggregate the results using Pandas
        result_all = (
            pd.DataFrame(result_list)
            .dropna()
            .groupby(['gene', 'gender', 'diplotype'])
            .sum()
            .reset_index()
            .sort_values(by="gene")
            .to_dict(orient='records')
        )            
        
        result_male = result_df[result_df['gender'] == 'male']
        result_female = result_df[result_df['gender'] == 'female']
        
        result_by_gender = {
            'all': result_all,
            'male': result_male.to_dict(orient='records'),
            'female': result_female.to_dict(orient='records')
        }
        
        return result_by_gender

In [6]:
def tran():
    data = count_diplotype(existing_data)

    df_all = pd.DataFrame(data['all'])
    df_all['gender'] = 'all'
    df_all['countkey'] = df_all['gender']+'-'+df_all['gene']+'-'+df_all['diplotype']
    df_all['totalkey'] = df_all['gender']+'-'+df_all['gene']
    df_all['lookupkey'] = df_all['gene']+'-'+df_all['diplotype']

    df_male = pd.DataFrame(data['male'])
    df_male['lookupkey'] = 'male'
    df_male['countkey'] = df_male['gender']+'-'+df_male['gene']+'-'+df_male['diplotype']
    df_male['totalkey'] = df_male['gender']+'-'+df_male['gene']
    df_male['lookupkey'] = df_male['gene']+'-'+df_male['diplotype']

    df_female = pd.DataFrame(data['female'])
    df_female['lookupkey'] = 'female'
    df_female['countkey'] = df_female['gender']+'-'+df_female['gene']+'-'+df_female['diplotype']
    df_female['totalkey'] = df_female['gender']+'-'+df_female['gene']
    df_female['lookupkey'] = df_female['gene']+'-'+df_female['diplotype']

    df = pd.concat([df_all, df_male, df_female], ignore_index=True)[['gender','countkey', 'totalkey','lookupkey', 'diplotype_count']].rename(columns={'gender': 'cou_gender'})

    df_count_dip = df.groupby('countkey', as_index=False)['diplotype_count'].sum().rename(columns={'diplotype_count': 'dip_cou'})

    df_count_total = df.groupby('totalkey', as_index=False)['diplotype_count'].sum().rename(columns={'diplotype_count': 'dip_total'})

    df_merge_cou = pd.merge(df, df_count_dip, how='inner', on='countkey')[['countkey','cou_gender', 'totalkey','lookupkey', 'dip_cou']].drop_duplicates().sort_values(by='lookupkey')
    df_merge_cou_total = pd.merge(df_merge_cou, df_count_total, how='inner', on='totalkey')[['countkey','cou_gender', 'totalkey','lookupkey', 'dip_cou', 'dip_total']].drop_duplicates().sort_values(by='lookupkey')

    master_data_path = '.out/master_data.json'

    result = []

    with open(master_data_path, 'r') as file:
        master_data = json.load(file)
        
    for i in master_data:   
        for j in i['gene_detail']:
            for k in j['diplotype']:
                master_result = {}
                master_result['sample_id'] = i['sample_id']
                master_result['nbt_id'] = i['nbt_id']
                master_result['gender'] = i['gender']
                master_result['gene'] = j['gene']
                master_result['diplotype'] = k
                master_result['lookupkey'] = j['gene']+'-'+k
                result.append(master_result)
                
    df_master = pd.DataFrame(result)

    df_merge = pd.merge(df_master, df_merge_cou_total, how='inner', on='lookupkey').sort_values(by=['sample_id', 'gene'])
    df_merge['dip_ratio'] = (df_merge['dip_cou']/df_merge['dip_total']).round(4)
    df_merge.fillna(0, inplace=True)

    pgx_count_diplotype = []

    # Get unique combinations of 'sample_id' and 'gene'
    unique_combinations = df_merge[['sample_id', 'gene']].drop_duplicates()

    for index, row in unique_combinations.iterrows():
        sample_id = row['sample_id']
        gene = row['gene']
        
        # Filter the DataFrame for the current 'sample_id' and 'gene' combination
        filtered_df = df_merge[(df_merge['sample_id'] == sample_id) & (df_merge['gene'] == gene)]
        
        # Get unique 'diplotype' values for the current combination
        unique_diplotypes = filtered_df['diplotype'].unique()
        
        for dip in unique_diplotypes:
            dict_result = {
                'cou_dip_sample_id': sample_id,
                'cou_dip_nbt_id': sample_id,  # Assuming this should also be 'sample_id'
                'cou_dip_gene': gene,
                'cou_dip_diplotype': dip,
                'cou_dip_gender_distribution': []
            }
            
            # Filter the DataFrame for the current 'sample_id', 'gene', and 'diplotype' combination
            filtered_dip_df = filtered_df[filtered_df['diplotype'] == dip]
            
            for index, dip_row in filtered_dip_df.iterrows():
                gender_distribution = {
                    'cou_gender': dip_row['cou_gender'],
                    'dip_cou': dip_row['dip_cou'],
                    'dip_total': dip_row['dip_total'],
                    'dip_ratio': dip_row['dip_ratio']
                }
                dict_result['cou_dip_gender_distribution'].append(gender_distribution)
            
            pgx_count_diplotype.append(dict_result)
    return pgx_count_diplotype

In [7]:
df_all = pd.DataFrame(data['all'])
df_all['gender'] = 'all'
df_all['countkey'] = df_all['gender']+'-'+df_all['gene']+'-'+df_all['diplotype']
df_all['totalkey'] = df_all['gender']+'-'+df_all['gene']
df_all['lookupkey'] = df_all['gene']+'-'+df_all['diplotype']

df_male = pd.DataFrame(data['male'])
df_male['lookupkey'] = 'male'
df_male['countkey'] = df_male['gender']+'-'+df_male['gene']+'-'+df_male['diplotype']
df_male['totalkey'] = df_male['gender']+'-'+df_male['gene']
df_male['lookupkey'] = df_male['gene']+'-'+df_male['diplotype']

df_female = pd.DataFrame(data['female'])
df_female['lookupkey'] = 'female'
df_female['countkey'] = df_female['gender']+'-'+df_female['gene']+'-'+df_female['diplotype']
df_female['totalkey'] = df_female['gender']+'-'+df_female['gene']
df_female['lookupkey'] = df_female['gene']+'-'+df_female['diplotype']

df = pd.concat([df_all, df_male, df_female], ignore_index=True)[['gender','countkey', 'totalkey','lookupkey', 'diplotype_count']].rename(columns={'gender': 'cou_gender'})
df.head()

,cou_gender,countkey,totalkey,lookupkey,diplotype_count
0,all,all-CACNA1S-Reference/Reference,all-CACNA1S,CACNA1S-Reference/Reference,7.0
1,all,all-CACNA1S-Reference/Reference,all-CACNA1S,CACNA1S-Reference/Reference,1.0
2,all,all-CFTR-ivacaftor non-responsive CFTR sequenc...,all-CFTR,CFTR-ivacaftor non-responsive CFTR sequence/iv...,7.0
3,all,all-CFTR-ivacaftor non-responsive CFTR sequenc...,all-CFTR,CFTR-ivacaftor non-responsive CFTR sequence/iv...,1.0
4,all,all-CYP2B6-*1/*1,all-CYP2B6,CYP2B6-*1/*1,1.0


In [8]:
# Create an empty list to collect the new rows
new_rows = []

list_gender = df['cou_gender'].drop_duplicates().to_list()
list_lookupkey = df['lookupkey'].drop_duplicates().to_list()

for key in list_lookupkey:
    list_filter_lookupkey = df[df['lookupkey'] == key]['cou_gender'].to_list()
    for gender in list_gender:
        if gender not in list_filter_lookupkey:
            new_row_dict = {
                'cou_gender': gender,
                'countkey': f'{gender}-{key}',
                'totalkey': f'{gender}-{str(key).split("-")[0]}',
                'lookupkey': key,
                'diplotype_count': float(0)
            }
            # Append the new row to the list
            new_rows.append(new_row_dict)

# Create a new DataFrame by concatenating the original DataFrame and the new rows
df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

In [9]:
df[df['lookupkey'] == 'CYP2B6-*1/*4']

,cou_gender,countkey,totalkey,lookupkey,diplotype_count
7,all,all-CYP2B6-*1/*4,all-CYP2B6,CYP2B6-*1/*4,1.0
180,female,female-CYP2B6-*1/*4,female-CYP2B6,CYP2B6-*1/*4,1.0
214,male,male-CYP2B6-*1/*4,male-CYP2B6,CYP2B6-*1/*4,0.0


In [10]:
# Group the DataFrame by the specified columns and count the occurrences
grouped = df.groupby(['lookupkey']).size().reset_index(name='count')

# Filter for rows with a count less than 3
filtered_rows = grouped[grouped['count'] < 3]

# Iterate through the filtered rows and print them
for index, row in filtered_rows.iterrows():
    lookupkey, count = row
    print(f"lookupkey: {lookupkey}, count: {count}")

In [11]:
df_count_total = df.groupby('totalkey', as_index=False)['diplotype_count'].sum().rename(columns={'diplotype_count': 'dip_total'})
df_count_total.head()

,totalkey,dip_total
0,all-CACNA1S,8.0
1,all-CFTR,8.0
2,all-CYP2B6,6.0
3,all-CYP2C19,8.0
4,all-CYP2C9,8.0


In [12]:
# df_cout_total = df.groupby(by='lookupkey')['gender', 'diplotype_count'].sum().reset_index().rename(columns={'diplotype_count':'cou_total'})
df_count_dip = df.groupby('countkey', as_index=False)['diplotype_count'].sum().rename(columns={'diplotype_count': 'dip_cou'})
df_count_dip.head()

,countkey,dip_cou
0,all-CACNA1S-Reference/Reference,8.0
1,all-CFTR-ivacaftor non-responsive CFTR sequenc...,8.0
2,all-CYP2B6-*1/*1,2.0
3,all-CYP2B6-*1/*4,1.0
4,all-CYP2B6-*1/*6,1.0


In [13]:
df_merge_cou = pd.merge(df, df_count_dip, how='inner', on='countkey')[['countkey','cou_gender', 'totalkey','lookupkey', 'dip_cou']].drop_duplicates().sort_values(by='lookupkey')
df_merge_cou_total = pd.merge(df_merge_cou, df_count_total, how='inner', on='totalkey')[['countkey','cou_gender', 'totalkey','lookupkey', 'dip_cou', 'dip_total']].drop_duplicates().sort_values(by='lookupkey')
df_merge_cou_total.head()

,countkey,cou_gender,totalkey,lookupkey,dip_cou,dip_total
0,all-CACNA1S-Reference/Reference,all,all-CACNA1S,CACNA1S-Reference/Reference,8.0,8.0
1,female-CACNA1S-Reference/Reference,female,female-CACNA1S,CACNA1S-Reference/Reference,7.0,7.0
2,male-CACNA1S-Reference/Reference,male,male-CACNA1S,CACNA1S-Reference/Reference,1.0,1.0
3,all-CFTR-ivacaftor non-responsive CFTR sequenc...,all,all-CFTR,CFTR-ivacaftor non-responsive CFTR sequence/iv...,8.0,8.0
4,female-CFTR-ivacaftor non-responsive CFTR sequ...,female,female-CFTR,CFTR-ivacaftor non-responsive CFTR sequence/iv...,7.0,7.0


In [14]:
# Group the DataFrame by the specified columns and count the occurrences
grouped = df_merge_cou.groupby(['lookupkey']).size().reset_index(name='count')

# Filter for rows with a count less than 3
filtered_rows = grouped[grouped['count'] < 3]

# Iterate through the filtered rows and print them
for index, row in filtered_rows.iterrows():
    lookupkey, count = row
    print(f"lookupkey: {lookupkey}, count: {count}")

In [15]:
# Group the DataFrame by the specified columns and count the occurrences
grouped = df_merge_cou_total.groupby(['lookupkey']).size().reset_index(name='count')

# Filter for rows with a count less than 3
filtered_rows = grouped[grouped['count'] < 3]

# Iterate through the filtered rows and print them
for index, row in filtered_rows.iterrows():
    lookupkey, count = row
    print(f"lookupkey: {lookupkey}, count: {count}")

In [16]:
master_data_path = '.out/master_data.json'

In [17]:
result = []

with open(master_data_path, 'r') as file:
    master_data = json.load(file)
    
for i in master_data:   
    for j in i['gene_detail']:
        for k in j['diplotype']:
            master_result = {}
            master_result['sample_id'] = i['sample_id']
            master_result['nbt_id'] = i['nbt_id']
            master_result['gender'] = i['gender']
            master_result['gene'] = j['gene']
            master_result['diplotype'] = k
            master_result['lookupkey'] = j['gene']+'-'+k
            result.append(master_result)


In [18]:
df_master = pd.DataFrame(result)
df_master.head()

,sample_id,nbt_id,gender,gene,diplotype,lookupkey
0,00023816,00023816,male,CACNA1S,Reference/Reference,CACNA1S-Reference/Reference
1,00023816,00023816,male,CFTR,ivacaftor non-responsive CFTR sequence/ivacaft...,CFTR-ivacaftor non-responsive CFTR sequence/iv...
2,00023816,00023816,male,CYP2B6,*1/*1,CYP2B6-*1/*1
3,00023816,00023816,male,CYP2C19,*1/*1,CYP2C19-*1/*1
4,00023816,00023816,male,CYP2C9,*1/*1,CYP2C9-*1/*1


In [19]:
df_merge = pd.merge(df_master, df_merge_cou_total, how='inner', on='lookupkey').sort_values(by=['sample_id', 'gene'])
df_merge['dip_ratio'] = df_merge['dip_cou']/df_merge['dip_total'].round(4)
df_merge.fillna(0, inplace=True)
df_merge.head()

,sample_id,nbt_id,gender,gene,diplotype,lookupkey,countkey,cou_gender,totalkey,dip_cou,dip_total,dip_ratio
9,00000039,00000039,female,CACNA1S,Reference/Reference,CACNA1S-Reference/Reference,all-CACNA1S-Reference/Reference,all,all-CACNA1S,8.0,8.0,1.0
10,00000039,00000039,female,CACNA1S,Reference/Reference,CACNA1S-Reference/Reference,female-CACNA1S-Reference/Reference,female,female-CACNA1S,7.0,7.0,1.0
11,00000039,00000039,female,CACNA1S,Reference/Reference,CACNA1S-Reference/Reference,male-CACNA1S-Reference/Reference,male,male-CACNA1S,1.0,1.0,1.0
33,00000039,00000039,female,CFTR,ivacaftor non-responsive CFTR sequence/ivacaft...,CFTR-ivacaftor non-responsive CFTR sequence/iv...,all-CFTR-ivacaftor non-responsive CFTR sequenc...,all,all-CFTR,8.0,8.0,1.0
34,00000039,00000039,female,CFTR,ivacaftor non-responsive CFTR sequence/ivacaft...,CFTR-ivacaftor non-responsive CFTR sequence/iv...,female-CFTR-ivacaftor non-responsive CFTR sequ...,female,female-CFTR,7.0,7.0,1.0


In [20]:
df_merge[(df_merge['dip_total'] < 8)&(df_merge['cou_gender'] == 'all')][['gene', 'dip_total']].sort_values(by='gene').groupby(by='gene').mean()

,dip_total
gene,
CYP2B6,6.0
CYP2D6,3.0


In [21]:
# Group the DataFrame by the specified columns and count the occurrences
grouped = df_merge.groupby(['sample_id', 'gene', 'diplotype']).size().reset_index(name='count')

# Filter for rows with a count less than 3
filtered_rows = grouped[grouped['count'] < 3]

# Iterate through the filtered rows and print them
for index, row in filtered_rows.iterrows():
    sample_id, gene, dip, count = row
    print(f"sample_id: {sample_id}, gene: {gene}, diplotype: {dip}, count: {count}")

In [22]:
pgx_count_diplotype = []

# Get unique combinations of 'sample_id' and 'gene'
unique_combinations = df_merge[['sample_id', 'gene']].drop_duplicates()

for index, row in unique_combinations.iterrows():
    sample_id = row['sample_id']
    gene = row['gene']
    
    # Filter the DataFrame for the current 'sample_id' and 'gene' combination
    filtered_df = df_merge[(df_merge['sample_id'] == sample_id) & (df_merge['gene'] == gene)]
    
    # Get unique 'diplotype' values for the current combination
    unique_diplotypes = filtered_df['diplotype'].unique()
    
    for dip in unique_diplotypes:
        dict_result = {
            'cou_dip_sample_id': sample_id,
            'cou_dip_nbt_id': sample_id,  # Assuming this should also be 'sample_id'
            'cou_dip_gene': gene,
            'cou_dip_diplotype': dip,
            'cou_dip_gender_distribution': []
        }
        
        # Filter the DataFrame for the current 'sample_id', 'gene', and 'diplotype' combination
        filtered_dip_df = filtered_df[filtered_df['diplotype'] == dip]
        
        for index, dip_row in filtered_dip_df.iterrows():
            gender_distribution = {
                'cou_gender': dip_row['cou_gender'],
                'dip_cou': dip_row['dip_cou'],
                'dip_total': dip_row['dip_total'],
                'dip_ratio': dip_row['dip_ratio']
            }
            dict_result['cou_dip_gender_distribution'].append(gender_distribution)
        
        pgx_count_diplotype.append(dict_result)


In [23]:
from pprint import pprint
pprint(pgx_count_diplotype[4])

{'cou_dip_diplotype': '*2/*2',
 'cou_dip_gender_distribution': [{'cou_gender': 'male',
                                  'dip_cou': 0.0,
                                  'dip_ratio': 0.0,
                                  'dip_total': 1.0},
                                 {'cou_gender': 'all',
                                  'dip_cou': 1.0,
                                  'dip_ratio': 0.125,
                                  'dip_total': 8.0},
                                 {'cou_gender': 'female',
                                  'dip_cou': 1.0,
                                  'dip_ratio': 0.14285714285714285,
                                  'dip_total': 7.0}],
 'cou_dip_gene': 'CYP2C19',
 'cou_dip_nbt_id': '00000039',
 'cou_dip_sample_id': '00000039'}


In [24]:
count = 0
count_totall = 0
for index in range(len(pgx_count_diplotype)):
    number_of_colection = len(pgx_count_diplotype[index]['cou_dip_gender_distribution'])
    if number_of_colection <= 2:
        # pprint(pgx_count_diplotype[['', '']].iloc[index])
        count += 1
    count_totall += 1
print(f'count = {count}')
print(f'count totall = {count_totall}')

count = 0
count totall = 142
